## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,53.53,95,98,8.90,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,82.27,91,100,11.50,moderate rain
2,2,Huilong,CN,31.8111,121.6550,83.89,78,96,4.61,overcast clouds
3,3,Albany,US,42.6001,-73.9662,82.81,47,74,6.60,broken clouds
4,4,Flinders,AU,-34.5833,150.8552,52.88,73,81,11.99,light rain
...,...,...,...,...,...,...,...,...,...,...
668,668,Bereda,ES,43.2686,-7.5406,71.15,88,99,1.95,overcast clouds
669,669,Dong Hoi,VN,17.4833,106.6000,78.58,88,100,5.26,overcast clouds
670,670,Tokomaru,NZ,-40.4667,175.5000,57.04,96,39,1.01,scattered clouds
671,671,Camacha,PT,33.0833,-16.3333,65.62,73,20,2.30,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,82.27,91,100,11.50,moderate rain
2,2,Huilong,CN,31.8111,121.6550,83.89,78,96,4.61,overcast clouds
3,3,Albany,US,42.6001,-73.9662,82.81,47,74,6.60,broken clouds
6,6,Myitkyina,MM,25.3833,97.4000,76.73,94,2,1.05,clear sky
10,10,Jamestown,US,42.0970,-79.2353,75.02,56,75,8.05,broken clouds
...,...,...,...,...,...,...,...,...,...,...
660,660,Diapaga,BF,12.0708,1.7889,79.16,71,100,5.41,overcast clouds
665,665,Waspan,NI,14.7419,-83.9717,80.13,86,84,6.67,broken clouds
667,667,Krasnyy Yar,RU,46.5331,48.3456,75.13,44,67,7.05,broken clouds
669,669,Dong Hoi,VN,17.4833,106.6000,78.58,88,100,5.26,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,82.27,moderate rain,19.7297,-155.0900,
2,Huilong,CN,83.89,overcast clouds,31.8111,121.6550,
3,Albany,US,82.81,broken clouds,42.6001,-73.9662,
6,Myitkyina,MM,76.73,clear sky,25.3833,97.4000,
10,Jamestown,US,75.02,broken clouds,42.0970,-79.2353,
14,Mantua,IT,78.04,scattered clouds,45.1667,10.7833,
15,Upata,VE,76.17,overcast clouds,8.0086,-62.3989,
16,Labuhan,ID,76.28,moderate rain,-6.8844,112.2051,
17,Butaritari,KI,82.54,broken clouds,3.0707,172.7902,
21,Lompoc,US,89.02,clear sky,34.6391,-120.4579,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # 6c. Get latitude and longitude from DataFrame

    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print(f"found hotel {hotels["results"][0]["name"]} in {hotel_df.loc[index, "City"]}")
        print(index)
    except (IndexError):
        print("hotel not found")
    except (TimeoutError):
        print("timeout")

1
2
hotel not found
6
10
14
15
16
17
21
23
24
30
32
34
35
36
40
hotel not found
43
50
51
58
61
64
70
85
89
91
93
103
104
106
109
116
117
118
119
125
128
133
139
140
147
154
155
158
161
166
168
169
174
176
178
179
hotel not found
188
193
194
197
199
200
203
205
207
209
211
212
216
218
224
227
231
233
hotel not found
247
249
251
253
257
259
260
261
265
266
268
270
271
272
277
278
282
289
292
294
299
301
304
305
310
311
313
314
315
316
317
319
320
324
325
327
328
334
335
337
340
343
346
348
350
351
352
354
357
358
362
363
371
372
374
378
385
386
387
388
392
398
402
405
407
410
hotel not found
413
417
418
419
420
421
422
425
426
428
431
434
436
437
438
448
452
453
456
457
462
463
hotel not found
473
475
476
hotel not found
484
491
492
502
503
506
508
513
515
519
520
522
526
531
533
536
538
539
540
541
544
547
552
553
558
hotel not found
565
567
568
575
581
584
585
588
590
591
592
hotel not found
594
hotel not found
597
598
601
605
613
619
622
623
624
627
629
634
636
639
641
642
646
648
650

In [42]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df_clean = hotel_df

# replace empty hotel names with NaN

hotel_df_clean['Hotel Name'].replace('', np.nan, inplace=True)

# drop null values

hotel_df_clean.dropna(inplace=True)

hotel_df_clean

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,82.27,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Huilong,CN,83.89,overcast clouds,31.8111,121.6550,Hilton Garden Inn Qidong
6,Myitkyina,MM,76.73,clear sky,25.3833,97.4000,Shwe Phyu Hotel
10,Jamestown,US,75.02,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
14,Mantua,IT,78.04,scattered clouds,45.1667,10.7833,Hotel la Favorita
...,...,...,...,...,...,...,...
660,Diapaga,BF,79.16,overcast clouds,12.0708,1.7889,Hôtel GACILIE
665,Waspan,NI,80.13,broken clouds,14.7419,-83.9717,Hostal casa de la rose
667,Krasnyy Yar,RU,75.13,broken clouds,46.5331,48.3456,Cottege near Ufa
669,Dong Hoi,VN,78.58,overcast clouds,17.4833,106.6000,Nam Long Hotel


In [53]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

hotel_df_clean.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_clean[["Lat", "Lng"]]

In [56]:
# create figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))